In [7]:
#QUESTION 1


import torch
import torch.nn.functional as F
image = torch.rand(6,6)
print("image=", image)
#Add a new dimension along 0th dimension
#i.e. (6,6) becomes (1,6,6). This is because
#pytorch expects the input to conv2D as 4d tensor
image = image.unsqueeze(dim=0)
print("image.shape=", image.shape)
image = image.unsqueeze(dim=0)
print("image.shape=", image.shape)
print("image=", image)
kernel = torch.ones(3,3)
#kernel = torch.rand(3,3)
print("kernel=", kernel)
kernel = kernel.unsqueeze(dim=0)
kernel = kernel.unsqueeze(dim=0)
#Perform the convolution
outimage = F.conv2d(image, kernel, stride=1, padding=0)
print("outimage=", outimage)

image= tensor([[0.8054, 0.7298, 0.2189, 0.0039, 0.6389, 0.3405],
        [0.6491, 0.0056, 0.0038, 0.9187, 0.2678, 0.2995],
        [0.0470, 0.1812, 0.7460, 0.6277, 0.4716, 0.5974],
        [0.0076, 0.3089, 0.8172, 0.5789, 0.1826, 0.5077],
        [0.1124, 0.8974, 0.3809, 0.4848, 0.3776, 0.2550],
        [0.1793, 0.7201, 0.1076, 0.8433, 0.0942, 0.0128]])
image.shape= torch.Size([1, 6, 6])
image.shape= torch.Size([1, 1, 6, 6])
image= tensor([[[[0.8054, 0.7298, 0.2189, 0.0039, 0.6389, 0.3405],
          [0.6491, 0.0056, 0.0038, 0.9187, 0.2678, 0.2995],
          [0.0470, 0.1812, 0.7460, 0.6277, 0.4716, 0.5974],
          [0.0076, 0.3089, 0.8172, 0.5789, 0.1826, 0.5077],
          [0.1124, 0.8974, 0.3809, 0.4848, 0.3776, 0.2550],
          [0.1793, 0.7201, 0.1076, 0.8433, 0.0942, 0.0128]]]])
kernel= tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
outimage= tensor([[[[3.3867, 3.4356, 3.8971, 4.1659],
          [2.7664, 4.1880, 4.6142, 4.4518],
          [3.4985, 5.0230, 4

In [3]:
#QUESTION 2

import torch
import torch.nn as nn

# Define input image
image = torch.rand(1, 1, 6, 6)  # Input shape: (batch_size, channels, height, width)

# Define convolutional layer with 3 output channels
conv_layer = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=3)

# Apply convolution
output = conv_layer(image)

print("Output shape using nn.Conv2d:", output.shape)


Output shape using nn.Conv2d: torch.Size([1, 3, 4, 4])


In [1]:
import torch
import torch.nn.functional as F

# Define kernel
kernel = torch.ones(3, 3)

# Add batch and channel dimensions to the kernel
kernel = kernel.unsqueeze(0).unsqueeze(0)
image = torch.rand(1, 1, 6, 6)  
# Perform convolution using functional API
output = F.conv2d(input=image, weight=kernel, stride=1, padding=0)

print("Output shape using functional API:", output.shape)


NameError: name 'image' is not defined

In [ ]:
#QUESTION 3

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transform
from torchvision.transforms import ToTensor
import torchvision.datasets as datasets
from torch.utils.data import Dataset,DataLoader

class CNNClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Conv2d(1,64,3),
                                 nn.ReLU(),
                                 nn.MaxPool2d((2,2),stride=2),
                                 nn.Conv2d(64,128,3),
                                 nn.ReLU(),
                                 nn.MaxPool2d((2,2),stride=2),
                                 nn.Conv2d(128,64,3),
                                 nn.ReLU(),
                                 nn.MaxPool2d((2,2),stride=2),
                                )
        self.classification_head = nn.Sequential(nn.Linear(64,20,bias=True),
                                                 nn.ReLU(),
                                                 nn.Linear(20,10,bias=True),)
        
    def forward(self,x):
        features = self.net(x)
        return self.classification_head(features.view(batch_size,-1))

mnist_trainset = datasets.MNIST(root="./data",download = True,train=True,transform=ToTensor())
train_loader = DataLoader(mnist_trainset,batch_size=50,shuffle=True)
mnist_testset = datasets.MNIST(root="./data",download = True,train=False,transform=ToTensor())
test_loader = DataLoader(mnist_testset,batch_size=50,shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNClassifier().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
batch_size=50

total_params = 0
for name,param in model.named_parameters():
    params = param.numel()
    total_params += params

for epoch in range(6):  
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:  
            print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print(f"Finished Training. Final loss = {loss.item()}, Total params = {total_params}")

correct,total = 0,0
for i,vdata in enumerate(test_loader):
    tinputs,tlabels = vdata[0].to(device), vdata[1].to(device)
    toutputs = model(tinputs)

    _,predicted = torch.max(toutputs,1)
    total += tlabels.size(0)
    correct += (predicted==tlabels).sum()
        
print(f"Correct = {correct}, Total = {total}")